In [2]:
library(data.table)
library(IRanges)

In [3]:
library(parallel)
options(mc.cores = detectCores())

In [4]:
real <- c('A*02:01', 'A*03:01', 'B*13:02', 'B*51:01', 'C*01:02', 'C*06:02', 'DRB1*07:01', 'DRB1*11:01', 'QB1*02:AB', 'DQB1*03:01', 'DPB1*02:01', 'DPB1*02:01')

In [70]:
dt <- fread('187521910.m8')

Read 5846112 rows and 12 (of 12) columns from 0.421 GB file in 00:00:06


In [71]:
setnames(dt, c('q', 't', 'iden', 'len', 'mis', 'gap', 'qf', 'qt', 'tf', 'tt', 'e', 'score'))
dt <- dt[iden == 100]
nexpr <- dt[grepl('\\D$', t)]
dt <- dt[grepl('\\d$', t)]
setkey(dt, q)


In [72]:
tt <- dt[, c('LEN', 'BEST') := .(max(len), paste(unique(sub('\\*.+', '', t[len == max(len)])), collapse = ',')), by = q]

In [76]:
table(dt$BEST)


             A            A,B          A,B,C      A,B,C,E,F            A,C 
         74377           7682          45724           2706          11047 
             B            B,C          B,C,E            B,F              C 
         35780          58437            760              5          41136 
   C,A,B,E,F,G      C,A,B,G,E      C,A,G,E,B            C,B            DMA 
          1637           1825            784           5198            503 
           DMB            DOA            DOB           DPA1           DPB1 
           723            581            996            649          12433 
          DQA1           DQB1      DQB1,DRB5            DRA           DRB1 
          1397           4584             38            428           9056 
     DRB1,DRB3 DRB1,DRB3,DRB4 DRB1,DRB3,DRB5      DRB1,DRB4      DRB1,DRB5 
          8499           1552           1516            191           1036 
          DRB3      DRB3,DRB1           DRB4           DRB5              E 
           

In [10]:
bad <- dt[len < LEN]
good <- dt[len == LEN]

In [11]:
nrow(good)

[1] 298298

In [12]:
nrow(bad)

[1] 64132

In [29]:
length(unique(good$t))

[1] 8973

In [30]:
length(unique(bad$t))

[1] 7632

In [31]:
best <- good[!t %in% bad$t]

In [33]:
length(unique(best$t))

[1] 1568

In [35]:
cands <- sub('-.+', '', unique(best$t))

In [54]:
temp <- bad[t == 'A*03:01-365']
sort(table(good[grepl('^A', t) & q %in% temp$q, t]))

A*01:37-365 A*02:250N-218  A*11:172-337 
            1             1             1

In [56]:
head(good[grepl('^A', t) & q %in% temp$q])

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,LEN
1,H5JCKCCXX:2:1:759613:0/1,A*01:37-365,100,28,0,0,148,65,252,279,4.9e-11,60.8,28
2,H5JCKCCXX:6:12:1110124:0/2,A*02:250N-218,100,21,0,0,1,63,186,206,2.4e-07,48.1,21
3,H5JCKCCXX:2:1:759613:0/1,A*11:172-337,100,28,0,0,148,65,252,279,4.9e-11,60.8,28


In [57]:
head(temp)

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,LEN
1,H5JCKCCXX:2:1:759613:0/1,A*03:01-365,100,27,0,0,145,65,253,279,1.9e-10,58.9,28
2,H5JCKCCXX:5:5:3023936:0/2,A*03:01-365,100,20,0,0,115,56,186,205,4.4e-07,47.4,22
3,H5JCKCCXX:6:12:1110124:0/2,A*03:01-365,100,20,0,0,2,61,187,206,5.4e-07,47,21


In [60]:
dt[q == 'H5JCKCCXX:2:1:759613:0/1' & grepl('03:01', t)]

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,LEN
1,H5JCKCCXX:2:1:759613:0/1,A*03:01-365,100,27,0,0,145,65,253,279,1.9e-10,58.9,28
2,H5JCKCCXX:2:1:759613:0/1,A*03:01:01:02N-319,100,27,0,0,145,65,253,279,1.9e-10,58.9,28


In [13]:
setkey(good, t)

In [14]:
acov <- good[grep('^A\\*', t), .(ir = sum(width(reduce(IRanges(tf, tt))))), by = t][order(-ir)]
print(head(acov))
acand <- acov[ir >= max(ir) * 0.85]
acand[sub('-.+', '', t) %in% real]

              t  ir
1:  A*02:01-365 297
2:  A*03:01-365 297
3: A*03:231-365 297
4:  A*03:26-365 297
5:  A*03:89-365 297
6: A*02:249-365 296


,t,ir
1,A*02:01-365,297
2,A*03:01-365,297


In [15]:
nrow(acand)

[1] 242

In [16]:
amatch <- good[t %in% acand$t]

In [17]:
nrow(amatch)

[1] 25727

In [ ]:
amatch[q %in% ]

In [18]:
aneg <- good[q %in% amatch$q & !(grepl('^A', t))]
nrow(aneg)

[1] 22920

In [22]:
nrow(aneg)

[1] 96

In [23]:
am <- dcast(amatch, q~t, value.var = 'len', fill = 0)
am <- am[, q := NULL]

In [24]:
nrow(am)

[1] 277

In [28]:
head(am)

,A*02:01-365,A*02:02-365,A*02:03-365,A*02:04-365,A*02:06-365,A*02:07-365,A*02:09-365,A*02:10-365,A*02:11-365,A*02:110-273,A*02:12-365,A*02:121-273,A*02:123-273,A*02:124-365,A*02:13-365,A*02:131-273,A*02:133-273,A*02:134-337,A*02:137-273,A*02:140-273,A*02:142-273,A*02:143-273,A*02:144-273,A*02:145-273,A*02:146-273,A*02:150-273,A*02:153-337,A*02:157-273,A*02:16-365,A*02:169-273,A*02:17-365,A*02:170-273,A*02:171-273,A*02:191-273,A*02:193-365,A*02:20-273,A*02:207-273,A*02:21-365,A*02:217-273,A*02:22-365,A*02:221-273,A*02:230-365,A*02:24-365,A*02:241-273,A*02:242-273,A*02:243-337,A*02:247-273,A*02:249-365,A*02:251-365,A*02:252-365,A*02:253-365,A*02:254-298,A*02:256-298,A*02:259-365,A*02:26-363,A*02:260-273,A*02:262-273,A*02:265-365,A*02:266-365,A*02:269-365,A*02:279-365,A*02:28-273,A*02:29-273,A*02:291-273,A*02:294-273,A*02:295-273,A*02:30-365,A*02:315-273,A*02:322-273,A*02:327-273,A*02:328-273,A*02:329-273,A*02:33-365,A*02:333-273,A*02:335-273,A*02:336-273,A*02:338-273,A*02:342-337,A*02:355-273,A*02:356N-255,A*02:358-273,A*02:369-273,A*02:370-273,A*02:372-298,A*02:374-357,A*02:390-337,A*02:393-273,A*02:394-273,A*02:397-273,A*02:40-273,A*02:403-273,A*02:404-337,A*02:405-273,A*02:406-273,A*02:407-273,A*02:409-337,A*02:41-273,A*02:411-273,A*02:415-273,A*02:416-273,A*02:418-273,A*02:419-337,A*02:42-273,A*02:426-337,A*02:427-273,A*02:428-337,A*02:436-339,A*02:437-273,A*02:441-273,A*02:442-337,A*02:443-337,A*02:444-337,A*02:445-337,A*02:446-337,A*02:447-337,A*02:448-337,A*02:449-337,A*02:450-337,A*02:451-337,A*02:452-337,A*02:453-337,A*02:454-273,A*02:455-365,A*02:463-273,A*02:464-273,A*02:465-337,A*02:467-337,A*02:469-337,A*02:470-337,A*02:471-337,A*02:472-337,A*02:473-337,A*02:474-337,A*02:475-337,A*02:477-337,A*02:478-337,A*02:479-337,A*02:48-365,A*02:480-337,A*02:481-273,A*02:482-273,A*02:488-273,A*02:493-337,A*02:506N-262,A*02:509-273,A*02:51-365,A*02:512-273,A*02:523-340,A*02:524-340,A*02:526-340,A*02:533-365,A*02:534-340,A*02:535-340,A*02:537-340,A*02:538-273,A*02:539-273,A*02:543-273,A*02:547-340,A*02:557-273,A*02:558-273,A*02:559-273,A*02:57-365,A*02:59-273,A*02:590-337,A*02:60-365,A*02:61-273,A*02:64-365,A*02:66-289,A*02:67-270,A*02:68-365,A*02:70-273,A*02:71-273,A*02:72-273,A*02:73-273,A*02:74-273,A*02:75-273,A*02:76-273,A*02:77-365,A*02:79-365,A*02:80-273,A*02:81-365,A*02:84-282,A*02:85-273,A*02:87-273,A*02:89-365,A*02:90-273,A*02:95-365,A*02:96-273,A*02:97-273,A*03:01-365,A*03:01:01:02N-319,A*03:02-365,A*03:03N-363,A*03:04-365,A*03:05-285,A*03:107-340,A*03:112-273,A*03:118-273,A*03:12-318,A*03:131-273,A*03:132-273,A*03:134-365,A*03:143-273,A*03:159-273,A*03:17-298,A*03:175-337,A*03:181-273,A*03:182-337,A*03:193-340,A*03:195-340,A*03:196-340,A*03:20-273,A*03:200Q-339,A*03:204-340,A*03:213-365,A*03:218-365,A*03:220-273,A*03:223-340,A*03:224-365,A*03:225-365,A*03:227-312,A*03:229-273,A*03:231-365,A*03:234Q-361,A*03:26-365,A*03:35-273,A*03:36N-356,A*03:37-337,A*03:39-365,A*03:42-273,A*03:43-273,A*03:44-273,A*03:45-273,A*03:48-340,A*03:55-273,A*03:72-273,A*03:77-365,A*03:78-365,A*03:89-365,A*11:25-365,A*30:89-365,A*69:01-365
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,0,NA,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA,NA,NA,NA,0,NA,NA,0,NA,0,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA

In [26]:
tail(sort(colSums(am)), 50)

named numeric(0)

In [27]:
sum(pmax((am[, 'A*03:01-365', with = F] + am[, 'A*02:01-365', with = F])))

[1] NA

In [30]:
sum(pmax((am[, 'A*03:26-365', with = F] + am[, 'A*02:96-273', with = F])))

[1] 11919

In [31]:
sum(pmax((am[, 'A*03:26-365', with = F] + am[, 'A*02:01-365', with = F])))

[1] 11722

In [26]:
sum(pmax((am[, 'A*03:89-365', with = F] + am[, 'A*02:01-365', with = F])))

[1] 11603

In [18]:
sum((am[, 'A*03:26-365', with = F] + am[, 'A*02:01-365', with = F]) > 0) / nrow(am)

[1] 0.8594249

In [19]:
sum((am[, 'A*03:89-365', with = F] + am[, 'A*02:01-365', with = F]) > 0) / nrow(am)

[1] 0.7795527

In [20]:
sum((am[, 'A*03:26-365', with = F] + am[, 'A*03:77-365', with = F]) > 0) / nrow(am)

[1] 0.600639

In [18]:
nr <- nrow(am)
am <- data.frame(am)
pairs <- mclapply(combn(colnames(am), 2, simplify = F), function(pair) sum((am[, pair[1]] + am[, pair[2]]) > 0) / nr)

In [153]:
?combn

combn {utils},R Documentation
x,"vector source for combinations, or integer n for x <- seq_len(n)."
m,number of elements to choose.
FUN,"function to be applied to each combination; default NULL means the identity, i.e., to return the combination (vector of length m)."
simplify,"logical indicating if the result should be simplified to an array (typically a matrix); if FALSE, the function returns a list. Note that when simplify = TRUE as by default, the dimension of the result is simply determined from FUN(1st combination) (for efficiency reasons). This will badly fail if FUN(u) is not of constant length."
...,"optionally, further arguments to FUN."
